# A3C Virtual Morris Water Maze

VMWM unity environment is a play ground to test navigation ability of patient with hippocampus trauma. Test subject is randomly rebornd and is to relocate the platform by using visual cues only after flags (show where the platform is under the water surface) are off.

The A.I. agent is to replicte similar behavior by using state-of-the-art deep learning model. 

![Maze.png](maze.png)

![Capture.PNG](http://upload-images.jianshu.io/upload_images/1873837-26d77968be99b564.PNG?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## Import Package

In [1]:
load_model = False # model's gonna use load_model as a global variable, so put it in front of model.
from model import *
import sys
import os
from inspect_checkpoints import print_tensors_in_checkpoint_file # Get it imported only for pre-trained models
checkpoints_dir = './tmp/checkpoints' # Get it only for pre-trained models

## VMWM Setup

In [2]:
max_episode_length = 200
gamma = .99 # discount rate for advantage estimation and reward discounting
s_size = 160*160 # Observations are greyscale frames of 84 * 84 * 1
a_size = 3 # Agent can move Left, Right, or Fire
load_model = False
model_path = './model'
noisy=True

## VMWM Train

In [3]:
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)

#Create a directory to save episode playback gifs to
if not os.path.exists('./frames'):
    os.makedirs('./frames')

with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    trainer = tf.train.AdamOptimizer(learning_rate=1e-4)
    master_network = AC_Network(s_size,a_size,'global',None,noisy,grayScale=gray) # Generate global network
    num_cpu = multiprocessing.cpu_count() # Set workers ot number of available CPU threads
    workers = []
        # Create worker classes
    for i in range(num_workers):
        worker = Worker(i,s_size,a_size,trainer,model_path,global_episodes,noisy,grayScale=gray)
        workers.append(worker)
        worker.start(setting=0)
    saver = tf.train.Saver(max_to_keep=5)

'''networks = ['global'] + ['worker_'+i for i in str(range(num_workers))]
print(networks)'''
#key = print_tensors_in_checkpoint_file('./tmp/checkpoints/mobilenet_v1_0.50_160.ckpt', tensor_name='',all_tensors=True)
#print(key)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        '''# Only for pre-trained models: https://github.com/tensorflow/models/tree/master/slim 
        dict = {}
        value = slim.get_model_variables('global'+'/MobilenetV1')
        for variable in value:
            name = variable.name.replace('global'+'/','').split(':')[0]
                #print(name)
            if name in key:
                dict[name] = variable
            #print(dict)
            #print(dict)
        init_fn = slim.assign_from_checkpoint_fn(
                            os.path.join(checkpoints_dir, 'mobilenet_v1_0.50_160.ckpt'),
                            dict)
        init_fn(sess)'''
        sess.run(tf.global_variables_initializer())

    # This is where the asynchronous magic happens.
    # Start the "work" process for each worker in a separate thread.
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(max_episode_length,gamma,sess,coord,saver)
        t = threading.Thread(target=(worker_work))
        t.start()
        sleep(0.5)
        worker_threads.append(t)
    coord.join(worker_threads)

local host--127.0.0.1:57192
local host--127.0.0.1:57199
local host--127.0.0.1:57203
local host--127.0.0.1:57207
local host--127.0.0.1:57215
Starting worker 0
Starting worker 1
Starting worker 2
Starting worker 3
Starting worker 4


C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00, 86.73it/s]


The connection was closed.
Doesn't read episode info.


Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-90bdd0304d88>", line 36, in <lambda>
    worker_work = lambda: worker.work(max_episode_length,gamma,sess,coord,saver)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\model.py", line 365, in work
    self.env.new_episode()
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\envVMWM.py", line 207, in new_episode
    self.s.send(bytes("PauseSpace" + END_TOKEN, 'utf8'))
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host



The connection was closed.
Doesn't read image as input.


...

Exception in thread Thread-14:
Traceback (most recent call last):
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-90bdd0304d88>", line 36, in <lambda>
    worker_work = lambda: worker.work(max_episode_length,gamma,sess,coord,saver)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\model.py", line 264, in work
    s = process_frame(s)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\helper.py", line 53, in process_frame
    s = scipy.misc.imresize(s,[84,84])
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\scipy\misc\pilutil.py", line 480, in imresize
    im = toimage(arr, mode=mode)
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\scipy\misc\pilutil.py", line 289, in toimage
    raise ValueError("'arr' does not have a suitable array sha

The connection was closed.
Doesn't read image as input.


Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-90bdd0304d88>", line 36, in <lambda>
    worker_work = lambda: worker.work(max_episode_length,gamma,sess,coord,saver)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\model.py", line 264, in work
    s = process_frame(s)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\helper.py", line 53, in process_frame
    s = scipy.misc.imresize(s,[84,84])
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\scipy\misc\pilutil.py", line 480, in imresize
    im = toimage(arr, mode=mode)
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\scipy\misc\pilutil.py", line 289, in toimage
    raise ValueError("'arr' does not have a suitable array sha

The connection was closed.
Doesn't read image as input.


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-90bdd0304d88>", line 36, in <lambda>
    worker_work = lambda: worker.work(max_episode_length,gamma,sess,coord,saver)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\model.py", line 264, in work
    s = process_frame(s)
  File "C:\Users\YuHang\Desktop\Water_Maze\v0.18\Analysis\helper.py", line 53, in process_frame
    s = scipy.misc.imresize(s,[84,84])
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\scipy\misc\pilutil.py", line 480, in imresize
    im = toimage(arr, mode=mode)
  File "C:\Users\YuHang\Anaconda2\envs\Maze\lib\site-packages\scipy\misc\pilutil.py", line 289, in toimage
    raise ValueError("'arr' does not have a suitable array sha